<a href="https://colab.research.google.com/github/nptan2005/spark401_colab/blob/main/notebookes/spark_level_2_join.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install -y openjdk-17-jdk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jre session-migration x11-utils
Suggested packages:
  gvfs libxt-doc openjdk-17-demo openjdk-17-source visualvm mesa-utils
The following NEW packages will be installed:
  at-spi2-core fonts-dejavu-core fonts-dejavu-extra gsettings-desktop-schemas
  libatk-bridge2.0-0 libatk-wrapper-java libatk-wrapper-java-jni libatk1.0-0
  libatk1.0-data libatspi2.0-0 libgail-common libgail18 libgtk2.0-0
  libgtk2.0-bin libgtk2.0-common librsvg2-common libxcomposite1 libxt-dev
  libxtst6 libxxf86dga1 openjdk-17-jdk openjdk-17-jr

In [3]:
!wget https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
!tar xf spark-4.0.1-bin-hadoop3.tgz

--2025-12-29 09:36:23--  https://archive.apache.org/dist/spark/spark-4.0.1/spark-4.0.1-bin-hadoop3.tgz
Resolving archive.apache.org (archive.apache.org)... 65.108.204.189, 2a01:4f9:1a:a084::2
Connecting to archive.apache.org (archive.apache.org)|65.108.204.189|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 548955321 (524M) [application/x-gzip]
Saving to: ‘spark-4.0.1-bin-hadoop3.tgz’

spark-4.0.1-bin-had 100%[===================>] 523.52M  10.5MB/s    in 32s     

2025-12-29 09:36:55 (16.3 MB/s) - ‘spark-4.0.1-bin-hadoop3.tgz’ saved [548955321/548955321]



In [4]:
# ===============================
# Spark 4.0.1 Setup (REQUIRED)
# ===============================
import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-4.0.1-bin-hadoop3"
os.environ["PATH"] += ":/content/spark-4.0.1-bin-hadoop3/bin"

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Spark401-Training") \
    .master("local[*]") \
    .config("spark.sql.shuffle.partitions", "4") \
    .getOrCreate()

print("Spark version:", spark.version)

Spark version: 4.0.1


# 🚀 Spark Level 2 – Bài 2: Filter, Select, WithColumn & Execution Plan

## 🎯 Mục tiêu bài này

**Sau bài này bạn sẽ:**
>*	Hiểu Transformation vs Action
>*	Biết Spark có chạy ngay hay không
>*	Bắt đầu chạm vào shuffle (rất quan trọng)

---

## 🧪 Bài toán

**Dữ liệu giao dịch:**

|order_id|customer_id|amount|country|
|--------|-----------|------|-------|
|1|C001|120|VN|
|2|C002|80|VN|
|3|C003|200|SG|
|4|C001|50|VN|




### 📌 Bước 1 – Tạo DataFrame:

In [5]:
from pyspark.sql import functions as F

data = [
    (1, "C001", 120, "VN"),
    (2, "C002", 80, "VN"),
    (3, "C003", 200, "SG"),
    (4, "C001", 50, "VN"),
]

columns = ["order_id", "customer_id", "amount", "country"]

df = spark.createDataFrame(data, columns)
df.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
|       2|       C002|    80|     VN|
|       3|       C003|   200|     SG|
|       4|       C001|    50|     VN|
+--------+-----------+------+-------+



### 📌 Bước 2 – Transformation (KHÔNG chạy ngay)

```python
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)
```

>❓ Spark đã chạy chưa?

>👉 CHƯA


In [6]:
df_vn = df.filter(df.country == "VN")
df_vn_high = df_vn.filter(df_vn.amount > 100)

### 📌 Bước 3 – Action (Spark bắt đầu chạy)

```python
df_vn_high.show()
```

#### 🧠 Nguyên lý quan trọng

|Loại|Ví dụ|
|----|-----|
|Transformation|filter, select, withColumn|
|Action|show, count, collect|

> 👉 Spark lazy execution


In [7]:
df_vn_high.show()

+--------+-----------+------+-------+
|order_id|customer_id|amount|country|
+--------+-----------+------+-------+
|       1|       C001|   120|     VN|
+--------+-----------+------+-------+



### 📌 Bước 4 – Thêm cột mới

In [8]:
df2 = df.withColumn(
    "amount_category",
    F.when(df.amount >= 100, "HIGH").otherwise("LOW")
)

df2.show()

+--------+-----------+------+-------+---------------+
|order_id|customer_id|amount|country|amount_category|
+--------+-----------+------+-------+---------------+
|       1|       C001|   120|     VN|           HIGH|
|       2|       C002|    80|     VN|            LOW|
|       3|       C003|   200|     SG|           HIGH|
|       4|       C001|    50|     VN|            LOW|
+--------+-----------+------+-------+---------------+



### 📌 Bước 5 – Xem Execution Plan:

```python
df2.explain(True)
```

Result:

```code
== Physical Plan ==
*(1) Project
+- *(1) Scan ExistingRDD
```

> 👉 Chưa có shuffle → nhẹ

In [9]:
df2.explain(True)

== Parsed Logical Plan ==
'Project [unresolvedstarwithcolumns(amount_category, CASE WHEN '`>=`(amount#2L, 100) THEN HIGH ELSE LOW END, None)]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Analyzed Logical Plan ==
order_id: bigint, customer_id: string, amount: bigint, country: string, amount_category: string
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= cast(100 as bigint)) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Optimized Logical Plan ==
Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- LogicalRDD [order_id#0L, customer_id#1, amount#2L, country#3], false

== Physical Plan ==
*(1) Project [order_id#0L, customer_id#1, amount#2L, country#3, CASE WHEN (amount#2L >= 100) THEN HIGH ELSE LOW END AS amount_category#30]
+- *(1) Scan ExistingRDD[order_id#0L,custom